<a href="https://colab.research.google.com/github/arthurziegler/transformers-for-NLP/blob/main/notebooks/Models_and_Tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 33.1 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 


In [2]:
from transformers import AutoTokenizer

In [3]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [5]:
tokenizer("hello world")

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [6]:
tokens = tokenizer.tokenize("hello world")
tokens

['hello', 'world']

In [7]:
ids = tokenizer.convert_tokens_to_ids(tokens)

In [8]:
ids

[7592, 2088]

In [9]:
tokenizer.convert_ids_to_tokens(ids)

['hello', 'world']

In [10]:
tokenizer.decode(ids)

'hello world'

In [11]:
ids = tokenizer.encode("hello world")
ids

[101, 7592, 2088, 102]

In [12]:
tokenizer.convert_ids_to_tokens(ids)

['[CLS]', 'hello', 'world', '[SEP]']

In [13]:
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

In [14]:
model_inputs = tokenizer("hello world")
model_inputs

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [16]:
data = [
    "I like cats.",
    "Do you like cats too?"
]
tokenizer(data)

{'input_ids': [[101, 1045, 2066, 8870, 1012, 102], [101, 2079, 2017, 2066, 8870, 2205, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [17]:
from transformers import AutoModelForSequenceClassification

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
outputs = model(**model_inputs)

AttributeError: ignored

In [20]:
model_inputs = tokenizer("hello world", return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 7592, 2088,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [21]:
# the default was to create a binary classifier
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.3294, -0.1830]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5965, -0.5294, -0.0646]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
outputs.logits

tensor([[-0.5965, -0.5294, -0.0646]], grad_fn=<AddmmBackward0>)

In [26]:
outputs['logits']

tensor([[-0.5965, -0.5294, -0.0646]], grad_fn=<AddmmBackward0>)

In [27]:
outputs[0]

tensor([[-0.5965, -0.5294, -0.0646]], grad_fn=<AddmmBackward0>)

In [29]:
# As a review let's suppose that we want to convert the output into a numpy array, which would be useful to calculate metrics like accuracy, f1-score and etc.
# First we use detach() because the tensor output also has a gradient on it, which we won't need
# Then we call cpu() to move the tensor from the GPU to the CPU. This is only needed if the model computation are happening in the GPU.
# Then we call numpy() to convert the tensor into a numpy array
outputs.logits.detach().cpu().numpy()

array([[-0.59650874, -0.5294473 , -0.06461298]], dtype=float32)

In [30]:
data = [
    "I like cats.",
    "Do you like cats too?"
]
model_inputs = tokenizer(data, return_tensors = 'pt')
model_inputs

ValueError: ignored

In [31]:
model_inputs = tokenizer(data, padding = True, truncation=True, return_tensors = 'pt')
model_inputs

{'input_ids': tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [33]:
model_inputs['input_ids']

tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]])

In [34]:
model_inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [35]:
outputs = model(**model_inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5623, -0.5485,  0.3581],
        [-0.5861, -0.5704,  0.2891]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)